<a href="https://colab.research.google.com/github/ComponentSoftTeam/Mistral-Kubectl-Instruct/blob/main/notebooks/kubectl_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install transformers peft bitsandbytes torch scipy pinecone-client sentence_transformers tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.3 MB/s eta 0:00:00


In [2]:
from transformers.generation.stopping_criteria import StoppingCriteria, StoppingCriteriaList
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import torch
import pinecone
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
from sentence_transformers.cross_encoder import CrossEncoder
import numpy as np
from torch import nn

## Set up semantic search

In [19]:
from google.colab import userdata

PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')

In [23]:
def get_embedding(text):
  embed_text = sentencetransformer_model.encode(text)
  vector_text = embed_text.tolist()

  return vector_text

In [5]:
def query_from_pinecone(query, top_k=3):
    # get embedding from THE SAME embedder as the documents
    query_embedding = get_embedding(query)

    return index.query(
      vector=query_embedding,
      top_k=top_k,
      include_metadata=True   # gets the metadata (dates, text, etc)
    ).get('matches')

In [6]:
def get_results_from_pinecone(query, top_k=3, re_rank=True, verbose=True):

    results_from_pinecone = query_from_pinecone(query, top_k=top_k)
    if not results_from_pinecone:
        return []

    if verbose:
        print("Query:", query)


    final_results = []

    if re_rank:
        if verbose:
            print('Document ID (Hash)\t\tRetrieval Score\tCE Score\tText')

        sentence_combinations = [[query, result_from_pinecone['metadata']['text']] for result_from_pinecone in results_from_pinecone]

        # Compute the similarity scores for these combinations
        similarity_scores = cross_encoder.predict(sentence_combinations, activation_fct=nn.Sigmoid())

        # Sort the scores in decreasing order
        sim_scores_argsort = reversed(np.argsort(similarity_scores))

        # Print the scores
        for idx in sim_scores_argsort:
            result_from_pinecone = results_from_pinecone[idx]
            final_results.append(result_from_pinecone)
            if verbose:
                print(f"{result_from_pinecone['id']}\t{result_from_pinecone['score']:.2f}\t{similarity_scores[idx]:.2f}\t{result_from_pinecone['metadata']['text'][:50]}")
        return final_results

    if verbose:
        print('Document ID (Hash)\t\tRetrieval Score\tText')
    for result_from_pinecone in results_from_pinecone:
        final_results.append(result_from_pinecone)
        if verbose:
            print(f"{result_from_pinecone['id']}\t{result_from_pinecone['score']:.2f}\t{result_from_pinecone['metadata']['text'][:50]}")

    return final_results

In [7]:
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2')

config.json:   0%|          | 0.00/791 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/134M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [8]:
pinecone_key = PINECONE_API_KEY

INDEX_NAME = 'k8s-semantic-search'
NAMESPACE = 'default'

In [9]:
pinecone.init(api_key=pinecone_key, environment="gcp-starter")

if not INDEX_NAME in pinecone.list_indexes():
    pinecone.create_index(
        INDEX_NAME,  # The name of the index
        dimension=768,  # The dimensionality of the vectors
        metric='cosine',  # The similarity metric to use when searching the index
        pod_type='starter'  # The type of Pinecone pod
    )

index = pinecone.Index(INDEX_NAME)

In [24]:
sentencetransformer_model = SentenceTransformer('sentence-transformers/multi-qa-mpnet-base-cos-v1')

In [11]:
def semantic_search(prompt):
    final_results = get_results_from_pinecone(prompt, top_k=3, re_rank=True, verbose=True)

    return 'First result:\n' + final_results[0]['metadata']['text'].replace('\n', ' ') + '\n' + 'Second result:\n' + final_results[1]['metadata']['text'].replace('\n', ' ') + '\n' + 'Third result:\n' + final_results[2]['metadata']['text'].replace('\n', ' ')

## Set up mistral

In [12]:
base_model_id = 'mistralai/Mistral-7B-Instruct-v0.1'
lora_model_id = 'ComponentSoft/mistral-kubectl-instruct'

In [13]:
tokenizer = AutoTokenizer.from_pretrained(
    lora_model_id,
    padding_side="left",
    add_eos_token=False,
    add_bos_token=True,
)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

In [14]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [15]:
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    use_cache=True,
    trust_remote_code=True,
)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/5.06G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
model = PeftModel.from_pretrained(base_model, lora_model_id)

model.eval()

In [17]:
stop_terms=["</s>", "#End"]
eos_token_ids_custom =  [torch.tensor(tokenizer.encode(term, add_special_tokens=False)).to("cuda") for term in stop_terms]

category_terms=["</s>", "\n"]
category_eos_token_ids_custom =  [torch.tensor(tokenizer.encode(term, add_special_tokens=False)).to("cuda") for term in stop_terms]


class EvalStopCriterion(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, score: torch.FloatTensor, **kwargs):
        return any(torch.equal(e, input_ids[0][-len(e):]) for e in eos_token_ids_custom)


class CategoryStopCriterion(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, score: torch.FloatTensor, **kwargs):
        return any(torch.equal(e, input_ids[0][-len(e):]) for e in category_eos_token_ids_custom)

start_template = '### Answer:'
command_template = '# Command:'
end_template = '#End'

In [51]:
def text_to_text_generation(prompt):
    prompt = prompt.strip()
    ''

    is_kubectl_prompt = (
        f"[INST] You are a helpful assistant who classifies prompts into three categories. Respond with 0 if it pertains to a 'kubectl' operation. This is an instruction that can be answered with a 'kubectl' action. Look for keywords like 'get', 'list', 'create', 'show', 'view', and other command-like words. This category is an instruction instead of a question. Respond with 1 only if the prompt is a question, and is about a definition related to Kubernetes, or non-action inquiries. Respond with 2 every other scenario, for example if the question is a general question, not related to Kubernetes or 'kubectl'.\n"
        f"So for instance the following:\n"
        f"List all pods in Kubernetes\n"
        f"Would get a response:\n"
        f"0 [/INST]"
        f'text: "{prompt}"'
        f'response (0/1/2): '
    )


    model_input = tokenizer(is_kubectl_prompt, return_tensors="pt").to("cuda")
    with torch.no_grad():
        response = tokenizer.decode(model.generate(**model_input, max_new_tokens=8, pad_token_id=tokenizer.eos_token_id, repetition_penalty=1.15, stopping_criteria=StoppingCriteriaList([CategoryStopCriterion()]))[0], skip_special_tokens=True)
    response = response[len(is_kubectl_prompt):]

    print('-----------------------------QUERY START-----------------------------')
    print('Prompt: ' + prompt)
    print('Classified as: ' + response)
    response_num = 2 # Default to generic question
    if '0' in response:
      response_num = 0
    elif '1' in response:
      response_num = 1


    # Check if general question
    if response_num == 0:
        prompt = f'[INST] {prompt}\n Lets think step by step. [/INST] {start_template}'
    elif response_num == 1:
        retrieved_results = semantic_search(prompt)
        print('Query:')
        print(f'[INST] You are an assistant who summarizes results retrieved from a book about Kubernetes. This summary should answer the question. If the answer is not in the retrieved results, use your general knowledge. [/INST] Question: {prompt}\nRetrieved results:\n{retrieved_results}\nResponse:')
        prompt = f'[INST] You are an assistant who summarizes results retrieved from a book about Kubernetes. This summary should answer the question. If the answer is not in the retrieved results, use your general knowledge. [/INST] Question: {prompt}\nRetrieved results:\n{retrieved_results}\nResponse:'
    else:
        prompt = f'[INST] {prompt} [/INST]'

    # Generate output
    model_input = tokenizer(prompt, return_tensors="pt").to("cuda")
    with torch.no_grad():
      response = tokenizer.decode(model.generate(**model_input, max_new_tokens=256, pad_token_id=tokenizer.eos_token_id, repetition_penalty=1.15, stopping_criteria=StoppingCriteriaList([EvalStopCriterion()]))[0], skip_special_tokens=True)

    # Get the relevalt parts
    start = response.index(start_template) + len(start_template) if start_template in response else len(prompt)
    start = response.index(command_template) + len(command_template) if command_template in response else start
    end = response.index(end_template) if end_template in response else len(response)
    true_response = response[start:end].strip()
    print('Returned: ' + true_response)
    print('------------------------------QUERY END------------------------------')

    return true_response

## Get results

In [52]:
res = text_to_text_generation('provide a detailed description of the cloud-node-4.cloud-provider.io node')
print('Response:\n' + res)

-----------------------------QUERY START-----------------------------
Prompt: provide a detailed description of the cloud-node-4.cloud-provider.io node
Classified as: 0
Returned: kubectl describe nodes cloud-node-4.cloud-provider.io
------------------------------QUERY END------------------------------
Response:
kubectl describe nodes cloud-node-4.cloud-provider.io
